[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hechtflorian/visual-anomaly-detection/blob/main/anomalib-benchmark.ipynb)


## Install Packages

- Install Anomalib
- Please run following cells to fix possible version problems

In [1]:
# Run the following cells if you need to install the requirements

!pip install anomalib[full]==1.2.0  # Anomalib version 1.2.0 was used for this benchmark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.6/486.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.4/219.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.0/726.0 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 19.6 MB/s et

In [1]:
!pip install matplotlib==3.6.0  # Fix possible version problems

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.6.0 which is incompatible.
bigframes 1.41.0 requires matplotlib>=3.7.1, but you have matplotlib 3.6.0 which is incompatible.


In [1]:
!pip install numpy==1.24.4  # Fix possible version problems

In [2]:
!pip uninstall -y ollama  # Run to fix anomalib.models import problem ("cannot import name '_encode_image' from 'ollama._client'")

Found existing installation: ollama 0.4.7
Uninstalling ollama-0.4.7:
  Successfully uninstalled ollama-0.4.7


## Main Benchmark

- please adapt the configurations in the beginning to your needs
- make sure to use GPU for best results
- for all the benchmarks, Google Colab & NVIDIA A100 GPU were used

In [15]:
# Anomalib Benchmark Notebook
# ===========================
# This notebook allows you to benchmark different anomaly detection models on various datasets with customizable settings.


#### Configuration Section (Modify these settings) ####
# ------------------------------------------------------------
### Choose your dataset:
# - "mvtec" (MVTec AD dataset)
# - "rubber_mats" (Dataspree Rubber Mats dataset)
# - "mvtec_multiclass" (Multi-class MVTec AD dataset)
DATASET = "mvtec"

### For MVTec AD, choose category:
# bottle/cable/capsule/carpet/grid/hazelnut/leather/metal_nut/pill/screw/tile/toothbrush/transistor/wood/zipper
CATEGORY = "bottle"  # Only used for MVTec dataset

### Image size (width, height)
# All Models were tested on: 256x256, 320x320, 448x448. For PatchCore-1% 360x360, 512x512 were also tested
# Multi-class Benchmark on MVTec AD with: 256x256
IMAGE_SIZE = (256, 256)

### Choose model:
# - "efficientad_s" (EfficientAD-S)
# - "efficientad_m" (EfficientAD-M)
# - "fastflow" (FastFlow)
# - "patchcore_default" (PatchCore-10%)
# - "patchcore_1percent" (PatchCore-1%)
MODEL = "fastflow"

# Number of epochs (for FastFlow) or steps (for EfficientAD)
MAX_EPOCHS = 100  # used for FastFlow
MAX_STEPS = 70000 # used for EfficientAD

# Batch Size Configuration
BATCH_SIZES = {
    "efficientad_s": {"train": 1, "eval": 32},
    "efficientad_m": {"train": 1, "eval": 32},
    "fastflow": {"train": 32, "eval": 32},
    "patchcore_default": {"train": 8, "eval": 8},
    "patchcore_1percent": {"train": 8, "eval": 8}
}

# Automatically set batch sizes based on selected model
TRAIN_BATCH_SIZE = BATCH_SIZES.get(MODEL, {}).get("train", 8)
EVAL_BATCH_SIZE = BATCH_SIZES.get(MODEL, {}).get("eval", 8)

print(f"=== Model Information ===")
print(f"Model: {MODEL}")
print(f"Train Batch Size: {TRAIN_BATCH_SIZE}")
print(f"Eval Batch Size: {EVAL_BATCH_SIZE}")

### Setup Dataset paths ###
# The Benchmark was done on MVTec AD, on Rubber Mats, and on MVTec AD in a multi-class setting
# If left empty, the datasets will be downloaded to default locations
# If you have the datasets already, specify the paths here
# for the benchmarks, you could use google drive to store the datasets

## MVTec AD dataset ##
MVTEC_ROOT = "/content/drive/MyDrive/MVTec"  # Will download automatically if empty

"""
## Rubber Mats dataset ##
# Needs to be downloaded manually: You can contact Data Spree GmbH for access to the Rubber Mats dataset! https://www.data-spree.com/de/kontakt
#RUBBER_MATS_ROOT = "./dataspree-rubber-mats-dataset-S/datasets/dataspree"

## Multi-class MVTec AD ##
# Needs to be downloaded and set up manually for now.
# Therefore, you can put together the 5 structures and 10 objects of MVTec AD into Train and Test Folders. Also, Please configure the Anomalib Folder according to your setup.
MVTEC_MULTICLASS_ROOT = "./MVTec-generalized/all" # Needs to be downloaded and set up manually for now.

# configure to match your folder structure
# you can set up 2 train and 2 test folders correspondig to objects and structures of MVTec AD
TRAIN_DIR = "train"
TEST_DIR = "test/defect"
NORMAL_TEST_DIR = "test/good"

"""

# Optional: Comet ML configuration
USE_COMET = False  # Set to True to enable Comet ML logging
COMET_API_KEY = "YOUR_API_KEY"  # Replace with your API key
PROJECT_NAME = "ad-benchmark"

# Optional imports for Comet ML
if USE_COMET:
    import comet_ml
    comet_ml.init(api_key=COMET_API_KEY)

# ------------------------------------------------------------


# Main imports
import os
import torch
import numpy as np
from pathlib import Path
import matplotlib
from matplotlib import pyplot as plt
import time
from tqdm import tqdm
import random
from pytorch_lightning import seed_everything

import anomalib
from anomalib.data import MVTec, Folder
from anomalib.models import Patchcore, EfficientAd, Fastflow
from anomalib.models.image.efficient_ad.torch_model import EfficientAdModelSize
from anomalib.engine import Engine
from anomalib import TaskType
from anomalib.data.utils import ValSplitMode, TestSplitMode
from anomalib.metrics import AUROC, AUPR, F1Score
from lightning.pytorch.callbacks import ModelCheckpoint
from torchvision.transforms.v2 import Compose, Resize, InterpolationMode, Normalize


# Check environment: Use GPU for training with best results!
print("\n=== Environment Information ===")
print("Anomalib version:", anomalib.__version__)
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda if hasattr(torch.version, 'cuda') else "Not available")
print("GPU availability:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Number of GPU devices:", torch.cuda.device_count())
    print("Name of current GPU:", torch.cuda.get_device_name(0))
    print(f"CUDA memory allocated: {torch.cuda.memory_allocated()}")
    print(f"CUDA memory reserved: {torch.cuda.memory_reserved()}")
print("Matplotlib version:", matplotlib.__version__)

# Set random seed
seed_everything(42) # Seed 42 was used for all Benchmarks
print("\n=== Random Seeds ===")
print(f"PyTorch seed: {torch.initial_seed()}")
print(f"NumPy seed: {np.random.get_state()[1][0]}")
print(f"Random seed: {random.getstate()[1][0]}")


"""
Can be uncommented and configured when access to rubber mats dataset is given.
# Import Rubber Mats dataset class if needed
if DATASET == "rubber_mats":
    print("\nImporting Rubber Mats dataset class...")
    rubber_mats_path = os.path.dirname(RUBBER_MATS_ROOT) if RUBBER_MATS_ROOT else "./dataspree-rubber-mats-dataset-S"
    sys.path.append(rubber_mats_path)
    try:
        from dataspree_anomalib_data import DataspreeDataModule
    except ImportError:
        print("ERROR: Could not import DataspreeDataModule. Please make sure the Rubber Mats dataset is downloaded.")
        print("The Dataspree Rubber Mats dataset must be downloaded manually from the official source.")
        raise
"""

transform = None

# Check if the model is PatchCore and dataset is Rubber Mats. If so, disable center-crop
if DATASET == "rubber_mats" and MODEL in ["patchcore_default", "patchcore_1percent"]:
    if not (MODEL == "patchcore_1percent" and IMAGE_SIZE == (512, 512)):  # image-size 512 too large to disable center-crop without performance drop
        transform = Compose([
            Resize(IMAGE_SIZE, antialias=True),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False),
    ])

# disable center-crop for PatchCore-1% on MVTec AD with 256x256 image size
if (DATASET == "mvtec" and MODEL == "patchcore_1percent" and IMAGE_SIZE == (256, 256)):
    transform = Compose([
        Resize(IMAGE_SIZE, antialias=True),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False),
    ])


### Setup Dataset ###
# ------------
print(f"\n=== Setting up {DATASET} dataset ===")

task = TaskType.CLASSIFICATION  # Only Classification was tested for the Benchmark. But you can choose Segmentation too.
seed = 42

# Load the appropriate dataset
if DATASET == "mvtec":
    print(f"Loading MVTec AD dataset, category: {CATEGORY}, image-size: {IMAGE_SIZE}")

    # If path is not provided, use default download location
    if not MVTEC_ROOT:
        MVTEC_ROOT = "./datasets/MVTec"
        print(f"No path provided for MVTec dataset. Will download to {MVTEC_ROOT}.")

    # MVTec will be automatically downloaded by anomalib if not found at the specified path
    datamodule = MVTec(
        root=MVTEC_ROOT,
        category=CATEGORY,
        image_size=IMAGE_SIZE,
        train_batch_size=TRAIN_BATCH_SIZE,
        eval_batch_size=EVAL_BATCH_SIZE,
        num_workers=2,
        task=task,
        seed=seed,
        transform=transform,  # transforms only used for PatchCore-1% and image-size 256x256
    )

    # ensure that MVTec AD dataset is there. If not it is automatically downloaded here.
    datamodule.prepare_data() # you can uncomment this line if download is not needed


elif DATASET == "rubber_mats":
    print(f"Loading Rubber Mats dataset, image-size: {IMAGE_SIZE}")
    if not RUBBER_MATS_ROOT:
        print("ERROR: Path for Rubber Mats dataset is required")
        print("The Dataspree Rubber Mats dataset must be downloaded manually from the official source.")
        raise ValueError("RUBBER_MATS_ROOT path must be specified")

    datamodule = DataspreeDataModule(
        ds_dataset_id=561,
        root=RUBBER_MATS_ROOT,
        task=task,
        train_batch_size=TRAIN_BATCH_SIZE,
        eval_batch_size=EVAL_BATCH_SIZE,
        image_size=IMAGE_SIZE,
        num_workers=2,
        seed=seed,
        transform=transform,  # transforms only used for PatchCore, to disable center-crop
    )


elif DATASET == "mvtec_multiclass":
    print("Loading Multi-class MVTec AD dataset")
    if not MVTEC_MULTICLASS_ROOT:
        print("ERROR: Path for MVTec Multiclass dataset is required")
        print("The MVTec Multiclass dataset must be downloaded manually.")
        raise ValueError("MVTEC_MULTICLASS_ROOT path must be specified")

    datamodule = Folder(
        name="MVTec-generalized",
        root=MVTEC_MULTICLASS_ROOT,
        normal_dir=TRAIN_DIR,
        abnormal_dir=TEST_DIR,
        normal_test_dir=NORMAL_TEST_DIR,
        task=task,
        train_batch_size=TRAIN_BATCH_SIZE,
        eval_batch_size=EVAL_BATCH_SIZE,
        num_workers=2,
        image_size=IMAGE_SIZE,
        seed=seed,
        val_split_mode=ValSplitMode.SAME_AS_TEST,
        normal_split_ratio=None,
    )


VALID_DATASETS = ["mvtec", "rubber_mats", "mvtec_multiclass"]
if DATASET not in VALID_DATASETS:
    raise ValueError(f"Unknown dataset: {DATASET}")


# Setup the datamodule
try:
    datamodule.setup()
    print("\n=== Dataset Information ===")
    print(f"Total training images: {len(datamodule.train_dataloader().dataset)}")
    print(f"Total validation images: {len(datamodule.val_dataloader().dataset)}")
    print(f"Total test images: {len(datamodule.test_dataloader().dataset)}")
except Exception as e:
    print(f"Error setting up dataset: {e}")
    if DATASET == "mvtec":
        print("\nTip: MVTec dataset will be automatically downloaded if not found.")
        print("If you're having connection issues, you can manually download the dataset from:")
        print("https://www.mvtec.com/company/research/datasets/mvtec-ad")
        print("and place it in the specified MVTEC_ROOT directory.")
    raise


### Setup Model ###
# ----------
print(f"\n=== Setting up {MODEL} model ===")

# Initialize the selected model
if MODEL == "efficientad_s":
    model = EfficientAd()
elif MODEL == "efficientad_m":
    model = EfficientAd(model_size=EfficientAdModelSize.M)
elif MODEL == "fastflow":
    model = Fastflow(backbone="wide_resnet50_2")
elif MODEL == "patchcore_default":
    model = Patchcore()
elif MODEL == "patchcore_1percent":
    model = Patchcore(
        backbone="wide_resnet101_2",
        layers=("layer2", "layer3"),
        pre_trained=True,
        coreset_sampling_ratio=0.01,
        num_neighbors=9,
    )
else:
    raise ValueError(f"Unknown model: {MODEL}")


# Setup Logger (optional)
# ---------------------
if USE_COMET:
    from anomalib.loggers import AnomalibCometLogger

    # Generate experiment name based on settings
    if DATASET == "mvtec":
        experiment_name = f"{MODEL}_{CATEGORY}_{IMAGE_SIZE[0]}"
    else:
        experiment_name = f"{MODEL}_{DATASET}_{IMAGE_SIZE[0]}"

    comet_logger = AnomalibCometLogger(
        project_name=PROJECT_NAME,
        experiment_name=experiment_name
    )
    logger = comet_logger
else:
    logger = None


# Setup Callbacks
# --------------
# Create results directory
results_dir = "./results"
os.makedirs(results_dir, exist_ok=True)

if MODEL in ["efficientad_s", "efficientad_m", "fastflow"]:
    # These models benefit from callbacks
    save_dir = f"{results_dir}/{MODEL}"
    if DATASET == "mvtec":
        save_dir += f"/MVTec/{CATEGORY}"
    else:
        save_dir += f"/{DATASET}"

    os.makedirs(save_dir, exist_ok=True)

    checkpoint_callback = ModelCheckpoint(
        monitor="image_AUROC",
        mode="max",
        dirpath=save_dir,
        save_top_k=1,
        save_last=True,
    )
else:
    # PatchCore doesn't need callbacks, and needs only 1 training epoch
    callbacks = None
    checkpoint_callback = None


### Setup Engine ###
# -----------
print("\n=== Setting up training engine ===")

if MODEL in ["efficientad_s", "efficientad_m"]:
  engine = Engine(
    task=task,
    callbacks=[checkpoint_callback],  #
    max_steps=MAX_STEPS,  # 70000 steps
    image_metrics={
        "Precision": {
            "class_path": "torchmetrics.Precision",
            "init_args": {"task": "binary"},
        },
        "Recall": {
            "class_path": "torchmetrics.Recall",
            "init_args": {"task": "binary"},
        },
        "AUROC": {
            "class_path": "torchmetrics.AUROC",
            "init_args": {"task": "binary"},
        },
        "F1Score": {
            "class_path": "torchmetrics.F1Score",
            "init_args": {"task": "binary"},
        },
        "AUPR": {
            "class_path": "torchmetrics.AveragePrecision",
            "init_args": {"task": "binary"},
        },
    },
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    logger=logger,
  )

elif MODEL == "fastflow":
  engine = Engine(
    task=task,
    callbacks=[checkpoint_callback], #
    max_epochs=MAX_EPOCHS, # 100 epochs
    image_metrics={
        "Precision": {
            "class_path": "torchmetrics.Precision",
            "init_args": {"task": "binary"},
        },
        "Recall": {
            "class_path": "torchmetrics.Recall",
            "init_args": {"task": "binary"},
        },
        "AUROC": {
            "class_path": "torchmetrics.AUROC",
            "init_args": {"task": "binary"},
        },
        "F1Score": {
            "class_path": "torchmetrics.F1Score",
            "init_args": {"task": "binary"},
        },
        "AUPR": {
            "class_path": "torchmetrics.AveragePrecision",
            "init_args": {"task": "binary"},
        },
    },
    accelerator="auto",
    devices=1,
    logger=logger,
    #log_every_n_steps=2,  # uncomment if logger is used, for more accurate logging of fastflow
  )

elif MODEL in ["patchcore_default", "patchcore_1percent"]:
  engine = Engine(
    task=task,
    image_metrics={
        "Precision": {
            "class_path": "torchmetrics.Precision",
            "init_args": {"task": "binary"},
        },
        "Recall": {
            "class_path": "torchmetrics.Recall",
            "init_args": {"task": "binary"},
        },
        "AUROC": {
            "class_path": "torchmetrics.AUROC",
            "init_args": {"task": "binary"},
        },
        "F1Score": {
            "class_path": "torchmetrics.F1Score",
            "init_args": {"task": "binary"},
        },
        "AUPR": {
            "class_path": "torchmetrics.AveragePrecision",
            "init_args": {"task": "binary"},
        },
    },
    accelerator="auto",
    devices=1,
    logger=logger,
  )

else:
    raise ValueError(f"Unknown model: {MODEL}")


### Train Model ###
# ----------
print("\n=== Training model ===")
try:
    engine.fit(model=model, datamodule=datamodule)

    # Print checkpoint information if applicable
    if MODEL in ["efficientad_s", "efficientad_m", "fastflow"] and checkpoint_callback:
        print(f"\nBest model saved: {checkpoint_callback.best_model_path}")
        print(f"Last model saved: {checkpoint_callback.last_model_path}")
except Exception as e:
    print(f"Error during training: {e}")
    raise

print(f"Transforms after Training: {datamodule.train_transform}")


### Test Model ###
# ---------
print("\n=== Testing model ===")

# Add safe globals for serialization, since pytorch changed load weights_only
torch.serialization.add_safe_globals([Compose])
torch.serialization.add_safe_globals([Resize])
torch.serialization.add_safe_globals([InterpolationMode])
torch.serialization.add_safe_globals([Normalize])
torch.serialization.add_safe_globals([EfficientAdModelSize])


if MODEL in ["efficientad_s", "efficientad_m", "fastflow"]:
    test_results = engine.test(model=model, datamodule=datamodule, ckpt_path=checkpoint_callback.best_model_path) # test only best model
    print("\nTest Results:")
    for metric_name, value in test_results[0].items():
          if isinstance(value, (int, float)):
              print(f"{metric_name}: {value:.3f}")


elif MODEL in ["patchcore_default", "patchcore_1percent"]:
    test_results = engine.test(model=model, datamodule=datamodule)
    print("\nTest Results:")
    for metric_name, value in test_results[0].items():
          if isinstance(value, (int, float)):
              print(f"{metric_name}: {value:.3f}")

else:
    raise ValueError(f"Unknown model: {MODEL}")


### Measure Model Efficiency ###
# ----------------------
if torch.cuda.is_available():
    print("\n=== Measuring model efficiency ===")

    # FPS and Latency measurement
    # batch-size=1 used for latency. 1000 num_runs were used, but you can lower them for demo.
    def measure_fps_latency(model, input_size=(1, 3, 256, 256), num_runs=100, exclude_first=10):
        model.eval().cuda()
        x = torch.rand(*input_size).cuda()
        time_list = []

        for _ in tqdm(range(num_runs), desc="Measuring FPS and Latency"):
            torch.cuda.synchronize()
            start_time = time.time()
            with torch.no_grad():
                model(x)
            torch.cuda.synchronize()
            time_list.append(time.time() - start_time)

        time_list = time_list[exclude_first:]
        if not time_list:
            raise ValueError(f"`exclude_first` ({exclude_first}) is too high for `num_runs` ({num_runs}).")

        latency_ms = (sum(time_list) / len(time_list)) * 1000
        fps = 1 / (sum(time_list) / len(time_list))

        return fps, latency_ms

    # Throughput measurement
    # batch-size=16 used for throughput, for all models. 1000 num_runs were used, but you can also lower them for demo.
    def measure_throughput(model, input_size=(16, 3, 256, 256), num_runs=100, batch_size=16, exclude_first=10):
        model.eval().cuda()
        x = torch.rand(*input_size).cuda()
        throughput_list = []

        for _ in tqdm(range(num_runs), desc="Measuring Throughput"):
            torch.cuda.synchronize()
            start_time = time.time()
            with torch.no_grad():
                model(x)
            torch.cuda.synchronize()
            throughput_list.append(time.time() - start_time)

        throughput_list = throughput_list[exclude_first:]
        throughput = (batch_size * len(throughput_list)) / sum(throughput_list)

        return throughput

    # Measure performance metrics (reduced number of runs demo. Use num_runs=1000 for proper results!)
    try:
        input_size = (1, 3, IMAGE_SIZE[0], IMAGE_SIZE[1])
        fps, latency_ms = measure_fps_latency(model, input_size=input_size, num_runs=100)
        fps = round(fps, 2)
        latency_ms = round(latency_ms, 2)

        input_size_throughput = (16, 3, IMAGE_SIZE[0], IMAGE_SIZE[1])
        throughput = measure_throughput(model, input_size=input_size_throughput, num_runs=100)
        throughput = round(throughput, 2)

        # Print results
        print("\n=== Performance Metrics ===")
        print(f"FPS: {fps}")
        print(f"Latency: {latency_ms} ms")
        print(f"Throughput: {throughput} images/second")

        # Log metrics to Comet if enabled
        if USE_COMET:
            comet_logger.experiment.log_metric(name="FPS", value=fps)
            comet_logger.experiment.log_metric(name="Latency(ms)", value=latency_ms)
            comet_logger.experiment.log_metric(name="Throughput(img/s)", value=throughput)

            # End experiment logging
            comet_logger.experiment.end()
    except Exception as e:
        print(f"Error measuring performance: {e}")
else:
    print("\nSkipping performance measurements as CUDA is not available")


print("\n=== Benchmark completed successfully ===")

INFO:lightning_fabric.utilities.seed:Seed set to 42


=== Model Information ===
Model: fastflow
Train Batch Size: 32
Eval Batch Size: 32

=== Environment Information ===
Anomalib version: 1.2.0
Torch version: 2.6.0+cu124
CUDA version: 12.4
GPU availability: True
Number of GPU devices: 1
Name of current GPU: Tesla T4
CUDA memory allocated: 1674863104
CUDA memory reserved: 1761607680
Matplotlib version: 3.6.0

=== Random Seeds ===
PyTorch seed: 42
NumPy seed: 42
Random seed: 2147483648

=== Setting up mvtec dataset ===
Loading MVTec AD dataset, category: bottle, image-size: (256, 256)

=== Dataset Information ===
Total training images: 209
Total validation images: 83
Total test images: 83

=== Setting up fastflow model ===

=== Setting up training engine ===

=== Training model ===


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/results/fastflow/MVTec/bottle exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | loss                  | FastflowLoss             | 0      | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_m

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/core/module.py:512: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUPR         │    0.9947184324264526     │
│        image_AUROC        │    0.9825396537780762     │
│       image_F1Score       │     0.949999988079071     │
│      image_Precision      │            1.0            │
│       image_Recall        │    0.9047619104385376     │
└───────────────────────────┴───────────────────────────┘


Test Results:
image_AUPR: 0.995
image_AUROC: 0.983
image_F1Score: 0.950
image_Precision: 1.000
image_Recall: 0.905

=== Measuring model efficiency ===


Measuring Throughput: 100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


=== Performance Metrics ===
FPS: 33.6
Latency: 29.76 ms
Throughput: 49.79 images/second

=== Benchmark completed successfully ===
